# 5.5 卷积神经网络(LeNet)

在[“多层感知机的从零开始实现”](../chapter03_deep-learning-basics/3.09_mlp-scratch.ipynb)一节里我们构造了一个含单隐藏层的多层感知机模型来对Fashion-MNIST数据集中的图像进行分类。每张图像高和宽均是28像素。我们将图像中的像素逐行展开，得到长度为784的向量，并输入进全连接层中。然而，这种分类方法有一定的局限性。

1. 图像在同一列邻近的像素在这个向量中可能相距较远。它们构成的模式可能难以被模型识别。
2. 对于大尺寸的输入图像，使用全连接层容易导致模型过大。假设输入是高和宽均为$1,000$像素的彩色照片（含3个通道）。即使全连接层输出个数仍是256，该层权重参数的形状也是$3,000,000\times 256$：它占用了大约3 GB的内存或显存。这会带来过于复杂的模型和过高的存储开销。

卷积层尝试解决这两个问题。一方面，卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性均可能被有效识别；另一方面，卷积层通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大。

卷积神经网络就是含卷积层的网络。本节里我们将介绍一个早期用来识别手写数字图像的卷积神经网络：LeNet [1]。这个名字来源于LeNet论文的第一作者Yann LeCun。LeNet展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。这个奠基性的工作第一次将卷积神经网络推上舞台，为世人所知。

----
![](../img/chapter05/5.5_lenet.png)

----
<center>LeNet网络结构</center>


## LeNet模型

LeNet分为卷积层块和全连接层块两个部分。下面我们分别介绍这两个模块。

卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用$5\times 5$的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为$2\times 2$，且步幅为2。由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。

卷积层块的输出形状为(批量大小, 通道, 高, 宽)。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

下面我们通过`Sequential`类来实现LeNet模型。

In [1]:
import os
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l

print(torch.__version__)

1.3.0+cpu


## 5.5.1 LeNet模型 

In [2]:
net = nn.Sequential()
net.add_module( 'C1',nn.Conv2d(1, 6, 5) ) #in_channels=1, out_channels=6, kernel_size=5
net.add_module( 'C1-activation', nn.Sigmoid() )
net.add_module( 'S2', nn.MaxPool2d(2, 2) ) #kernel_size=2, stride=2
net.add_module( 'C3', nn.Conv2d(6, 16, 5)  )
net.add_module( 'C3-activation', nn.Sigmoid() )
net.add_module( 'S4', nn.MaxPool2d(2, 2) ) #kernel_size=2, stride=2
net.add_module( 'S4-flatten', nn.Flatten() )
net.add_module( 'C5', nn.Linear(16*4*4, 120) )
net.add_module( 'C5-activation', nn.Sigmoid() )
net.add_module( 'F6', nn.Linear(120, 84) )
net.add_module( 'F6-activation', nn.Sigmoid() )
net.add_module( 'OUTPUT', nn.Linear(84, 10) )

接下来查看每个层的形状。

In [4]:
print(net)

Sequential(
  (C1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (C1-activation): Sigmoid()
  (S2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (C3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (C3-activation): Sigmoid()
  (S4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (S4-flatten): Flatten()
  (C5): Linear(in_features=256, out_features=120, bias=True)
  (C5-activation): Sigmoid()
  (F6): Linear(in_features=120, out_features=84, bias=True)
  (F6-activation): Sigmoid()
  (OUTPUT): Linear(in_features=84, out_features=10, bias=True)
)


可以看到，在卷积层块中输入的高和宽在逐层减小。卷积层由于使用高和宽均为5的卷积核，从而将高和宽分别减小4，而池化层则将高和宽减半，但通道数则从1增加到16。全连接层则逐层减少输出个数，直到变成图像的类别数10。

## 5.5.2 获取数据和训练模型

下面我们来实验LeNet模型。实验中，我们仍然使用Fashion-MNIST作为训练数据集。

In [6]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

准备好训练集和测试集之后，先查看一下。☟

In [7]:
print( '6维张量的第一行数值：', list(train_iter)[0][0][0][0][0] ) # 235×2×256×1×28×28
print( '28×28图像的第一行元素的个数：', len(list(train_iter)[0][0][0][0][0]) )
test_X = list(train_iter)[0][0] #取出第一批样本，２５６×１×２８×２８
print( '第一批样本的形状：', test_X.size() )
test_y_hat = net(test_X)
print( '第一个前向输出的形状：', test_y_hat.size() )

6维张量的第一行数值： tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0078, 0.0000,
        0.0000, 0.0000, 0.0824, 0.0000, 0.0000, 0.0000, 0.0000, 0.0196, 0.1020,
        0.0000, 0.0000, 0.0000, 0.0078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])
28×28图像的第一行元素的个数： 28
第一批样本的形状： torch.Size([256, 1, 28, 28])
第一个前向输出的形状： torch.Size([256, 10])


因为卷积神经网络计算比多层感知机要复杂，建议使用GPU来加速计算。因此，我们对3.6节（softmax回归的从零开始实现）中描述的`evaluate_accuracy`函数略作修改，使其支持GPU计算。

In [8]:
device = d2l.get_current_device()
print(device)

cpu


相应地，我们对[“softmax回归的从零开始实现”](../chapter03_deep-learning-basics/3.06_softmax-regression-scratch.ipynb)一节中描述的`evaluate_accuracy`函数略作修改。由于数据刚开始存在CPU使用的内存上，当`ctx`变量代表GPU及相应的显存时，我们通过[“GPU计算”](../chapter04_deep-learning_computation/4.6_use-gpu.ipynb)一节中介绍的`Tensor.to('cuda')`函数将数据复制到显存上，例如`cuda:0`。

In [9]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进：它的完整实现将在“图像增广”一节中描述
def evaluate_accuracy(data_iter, net):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

我们同样对[“softmax回归的从零开始实现”](../chapter03_deep-learning-basics/3.06_softmax-regression-scratch.ipynb)一节中定义的`train_ch3`函数略作修改，确保计算使用的数据和模型同在内存或显存上。

In [10]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

我们重新将模型参数初始化到设备变量`device`之上。损失函数和训练算法则依然使用交叉熵损失函数和小批量随机梯度下降。

In [11]:
lr, num_epochs = 0.9, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [12]:
start = time.time()
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)
#train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)
print(time.time() - start)

training on  cpu


KeyboardInterrupt: 

## 小结

* 卷积神经网络就是含卷积层的网络。
* LeNet交替使用卷积层和最大池化层后接全连接层来进行图像分类。

## 练习

* 尝试基于LeNet构造更复杂的网络来提高分类准确率。例如，调整卷积窗口大小、输出通道数、激活函数和全连接层输出个数。在优化方面，可以尝试使用不同的学习率、初始化方法以及增加迭代周期。

## 参考文献

[1] LeCun, Y., Bottou, L., Bengio, Y., & Haffner, P. (1998). Gradient-based learning applied to document recognition. Proceedings of the IEEE, 86(11), 2278-2324.


## 扫码直达[知乎专栏](https://zhuanlan.zhihu.com/unicom-d2l)

![](../img/zhihu.png)